<div style="background: #000;
            color: #FFF;
            margin: 0px;
            padding: 10px 0px 20px 0px;
            text-align: center; 
                ">
    <h1>Week 18 Lab</h1>
</div>

For today's lab, we'll be using the Marvel comic book characters data set available from [538](https://github.com/fivethirtyeight/data/tree/master/comic-characters)

The data comes from [Marvel Wikia](http://marvel.wikia.com/Main_Page) and [DC Wikia](http://dc.wikia.com/wiki/Main_Page). Characters were scraped on August 24. Appearance counts were scraped on September 2. The month and year of the first issue each character appeared in was pulled on October 6.

The data is split into two files, for DC and Marvel, respectively: `dc-wikia-data.csv` and `marvel-wikia-data.csv`. Each file has the following variables:

Variable | Definition
---|---------
`page_id` | The unique identifier for that characters page within the wikia
`name` | The name of the character
`urlslug` | The unique url within the wikia that takes you to the character
`ID` | The identity status of the character (Secret Identity, Public identity, [on marvel only: No Dual Identity])
`ALIGN` | If the character is Good, Bad or Neutral
`EYE` | Eye color of the character
`HAIR` | Hair color of the character
`SEX` | Sex of the character (e.g. Male, Female, etc.)
`GSM` | If the character is a gender or sexual minority (e.g. Homosexual characters, bisexual characters)
`ALIVE` | If the character is alive or deceased
`APPEARANCES` | The number of appareances of the character in comic books (as of Sep. 2, 2014. Number will become increasingly out of date as time goes on.)
`FIRST APPEARANCE` | The month and year of the character's first appearance in a comic book, if available
`YEAR` | The year of the character's first appearance in a comic book, if available


Importing the libraries we'll be using

In [3295]:
import numpy as np 
import pandas as pd 
import datetime

In [3296]:
#Increase column width 
pd.options.display.max_colwidth = 100 #'Spider-Man (Peter Parker)'

Let's open the `marvel-wikia-data.csv` file and generate a dataframe. Then we'll check the top of the `df` to see what we have.

In [3297]:
df = pd.read_csv("../datasets/marvel-wikia-data.csv")
df.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950.0


As you can see we have a couple of issues:
* The column `names` has, in some cases, both the hero name and their secret identity. In other cases, it has the universe the hero is from (usually Earth-616). We'd like for `name` to be the superhero. If a secret identity name is available, we'd like for that to be in it's own column. We'd also like a column `origin` for the superhero's place of origin (usually Earth-616).
* The `urlslug` column is incorrect. We'd like for that to be a complete link. For example, spiderman's wikia link is https://marvel.fandom.com/wiki/Peter_Parker_(Earth-616) but if you use the `Spider-Man_(Peter_Parker)` from the `urlslug` (https://marvel.fandom.com/wiki/Spider-Man_(Peter_Parker), the page redirects to the correct page. You can choose to fix it various different ways but it should be a working link. Use "https://marvel.fandom.com/wiki/" as the beginning of your url.
* For the column `align`, we'd like for the "Characters" part to be removed. We'd like for that column to be categorical (Good, Neutral, Evil or blank).
* Same for the columns `EYE`, `HAIR`, `SEX` and `ID`. We want to drop the extraneous data and have simple categories (e.g. for `ID` we want categories ["Known","Public", "Secret", "None"] and for `SEX` we just want ["Male", "Female","Genderfluid","Agender"]. There's a way shown below that shows the various possible categories.
* Some data is in the wrong format. For example, `Year` and `APPEARANCES` are floats but they really should be ints (as you cannot have a non-int number of appearances). 
* Some data doesn't match up (like the `FIRST APPEARANCE` and `Year` columns might have not be the same year)
* Some columns are unnecessary. Some are missing data. Some rows are duplicates. Fix those as necessary.
* Some column names are written as uppercase, some are lowercase and some are mixed case. Normalize them so all the column names are in the same format.

Fix as many of these issues as you're able to. At the minimum, complete 4 of the above bulletpoints.

In [3298]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16376 entries, 0 to 16375
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   page_id           16376 non-null  int64  
 1   name              16376 non-null  object 
 2   urlslug           16376 non-null  object 
 3   ID                12606 non-null  object 
 4   ALIGN             13564 non-null  object 
 5   EYE               6609 non-null   object 
 6   HAIR              12112 non-null  object 
 7   SEX               15522 non-null  object 
 8   GSM               90 non-null     object 
 9   ALIVE             16373 non-null  object 
 10  APPEARANCES       15280 non-null  float64
 11  FIRST APPEARANCE  15561 non-null  object 
 12  Year              15561 non-null  float64
dtypes: float64(2), int64(1), object(10)
memory usage: 1.6+ MB


In [3299]:
def see_possible_categories(col):
    return df.groupby(col).sum()
    
see_possible_categories('name')

,page_id,APPEARANCES,Year
name,,,
'Spinner (Earth-616),322600,0.0,2007.0
107 (Earth-616),116257,1.0,1994.0
11-Ball (Earth-616),543479,1.0,1991.0
115 (Legion Personality) (Earth-616),624448,1.0,2010.0
14 (Earth-616),116253,1.0,1994.0
...,...,...,...
Zzxz (Earth-616),250460,6.0,2009.0
Zzzax (Earth-616),1553,32.0,1973.0
"\""Spider-Girl\"" (Mutant\/Spider Clone) (Earth-616)",689359,0.0,2013.0


* The column names has, in some cases, both the hero name and their secret identity. In other cases, it has the universe the hero is from (usually Earth-616). We'd like for name to be the superhero. If a secret identity name is available, we'd like for that to be in it's own column. We'd also like a column origin for the superhero's place of origin (usually Earth-616).

In [3300]:
df["origin"] = df['name'].str.split("(",2).str[2].str.split(")",1).str[0]
df["origin"]

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
16371          NaN
16372    Earth-616
16373    Earth-616
16374    Earth-616
16375          NaN
Name: origin, Length: 16376, dtype: object

In [3301]:
def see_possible_categories(col):
    return df.groupby(col).sum()
    
see_possible_categories("origin")

,page_id,APPEARANCES,Year
origin,,,
Earth-616,794183741,14846.0,4395394.0
Earth-7888,62360,0.0,1993.0
Mania,670753,23.0,2003.0


In [3302]:
## Superhero Name. I like Ironman
#df["name"] = df["name"].str.split('(',1).str[0].str.split(")",1).str[-2]
#df['name'].str.split(' ').str[2]

print(df["name"][3])

Iron Man (Anthony \"Tony\" Stark)


In [3358]:
df["SecretIdentity"] = df["name"].str.split('),', = 2, expand = False)
#.str[-1].str.split('(',1).str[2].str.split(")",2).str[-3]
#df["secret_identity"] = df['name'].str.split('(',3).str[1]

# dropping null value columns to avoid errors 
#df.dropna(inplace = True) 
   
# new data frame with split value columns 
#data["Team"]= data["Team"].str.split("t", n = 1, expand = True) 
df["SecretIdentity"]

SyntaxError: invalid syntax (<ipython-input-3358-def217b1c1d6>, line 1)

In [3304]:
df[df["secretidentity"].notnull()]

KeyError: 'secretidentity'

In [2963]:
#To reorganize the columns
df.columns

Index(['page_id', 'name', 'urlslug', 'ID', 'ALIGN', 'EYE', 'HAIR', 'SEX',
       'GSM', 'ALIVE', 'APPEARANCES', 'FIRST APPEARANCE', 'Year', 'origin',
       'SecretIdentity'],
      dtype='object')

In [2964]:
df = df.reindex(['page_id',
 'name',
 'SecretIdentity',
 'origin',
 'urlslug',
 'ID',
 'ALIGN',
 'EYE',
 'HAIR',
 'SEX',
 #'GSM', To remove GSM
 'ALIVE',
 'APPEARANCES',
 'FIRST APPEARANCE',
 'Year'], axis=1)

df

,page_id,name,SecretIdentity,origin,urlslug,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man,Spider-Man,NaN,\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America,Captain America,NaN,\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,Living Characters,3360.0,Mar-41,1941.0
2,64786,Wolverine,Wolverine,NaN,\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,3061.0,Oct-74,1974.0
3,1868,Iron Man,Iron Man,NaN,\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor,Thor,NaN,\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,Living Characters,2258.0,Nov-50,1950.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach,Ru'ach,NaN,\/Ru%27ach_(Earth-616),No Dual Identity,Bad Characters,Green Eyes,No Hair,Male Characters,Living Characters,NaN,NaN,NaN
16372,665474,Thane,Thane,Earth-616,\/Thane_(Thanos%27_son)_(Earth-616),No Dual Identity,Good Characters,Blue Eyes,Bald,Male Characters,Living Characters,NaN,NaN,NaN
16373,695217,Tinkerer,Tinkerer,Earth-616,\/Tinkerer_(Skrull)_(Earth-616),Secret Identity,Bad Characters,Black Eyes,Bald,Male Characters,Living Characters,NaN,NaN,NaN
16374,708811,TK421,TK421,Earth-616,\/TK421_(Spiderling)_(Earth-616),Secret Identity,Neutral Characters,NaN,NaN,Male Characters,Living Characters,NaN,NaN,NaN


* The urlslug column is incorrect. We'd like for that to be a complete link. For example, spiderman's wikia link is https://marvel.fandom.com/wiki/PeterParker(Earth-616) but if you use the Spider-Man_(Peter_Parker) from the urlslug (https://marvel.fandom.com/wiki/Spider-Man_(Peter_Parker), the page redirects to the correct page. You can choose to fix it various different ways but it should be a working link. Use "https://marvel.fandom.com/wiki/" as the beginning of your url.

In [2965]:

#changint the display so that the link appears complete and we can click it to open in browser
#df["urlslug"]= df["urlslug"].str.split("_").str[0]
df["urlslug"] = "http://marvel.fandom.com/wiki/" + df["urlslug"].str.split("/").str[1].replace(["   "], "")
#df["urlslug"]= df["urlslug"].str[:][1]
df["urlslug"]
df


,page_id,name,SecretIdentity,origin,urlslug,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man,Spider-Man,NaN,http://marvel.fandom.com/wiki/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,Living Characters,4043.0,Aug-62,1962.0
1,7139,Captain America,Captain America,NaN,http://marvel.fandom.com/wiki/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,Living Characters,3360.0,Mar-41,1941.0
2,64786,Wolverine,Wolverine,NaN,http://marvel.fandom.com/wiki/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,3061.0,Oct-74,1974.0
3,1868,Iron Man,Iron Man,NaN,http://marvel.fandom.com/wiki/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Living Characters,2961.0,Mar-63,1963.0
4,2460,Thor,Thor,NaN,http://marvel.fandom.com/wiki/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,Living Characters,2258.0,Nov-50,1950.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach,Ru'ach,NaN,http://marvel.fandom.com/wiki/Ru%27ach_(Earth-616),No Dual Identity,Bad Characters,Green Eyes,No Hair,Male Characters,Living Characters,NaN,NaN,NaN
16372,665474,Thane,Thane,Earth-616,http://marvel.fandom.com/wiki/Thane_(Thanos%27_son)_(Earth-616),No Dual Identity,Good Characters,Blue Eyes,Bald,Male Characters,Living Characters,NaN,NaN,NaN
16373,695217,Tinkerer,Tinkerer,Earth-616,http://marvel.fandom.com/wiki/Tinkerer_(Skrull)_(Earth-616),Secret Identity,Bad Characters,Black Eyes,Bald,Male Characters,Living Characters,NaN,NaN,NaN
16374,708811,TK421,TK421,Earth-616,http://marvel.fandom.com/wiki/TK421_(Spiderling)_(Earth-616),Secret Identity,Neutral Characters,NaN,NaN,Male Characters,Living Characters,NaN,NaN,NaN


* For the column `align`, we'd like for the "Characters" part to be removed. We'd like for that column to be categorical (Good, Neutral, Evil or blank).

In [2966]:
df["ALIGN"] = df["ALIGN"].str.split('C',1).str[0]
df["ALIGN"] 

0           Good 
1           Good 
2        Neutral 
3           Good 
4           Good 
           ...   
16371        Bad 
16372       Good 
16373        Bad 
16374    Neutral 
16375        Bad 
Name: ALIGN, Length: 16376, dtype: object

In [2967]:
def see_possible_categories(col):
    return df.groupby(col).sum()
    
see_possible_categories("ALIGN")

,page_id,APPEARANCES,Year
ALIGN,,,
Bad,2030768533,52910.0,12780912.0
Good,1310680907,150603.0,8670851.0
Neutral,771906748,41091.0,4122737.0


* Same for the columns EYE, HAIR, SEX and ID. We want to drop the extraneous data and have simple categories (e.g. for ID we want categories "Known","Public", "Secret", "None" and for SEX we just want "Male", "Female","Genderfluid","Agender". There's a way shown below that shows the various possible categories.

In [2968]:
df["EYE"] = df["EYE"].str.split(' ').str[0]
df["EYE"] 

0        Hazel
1         Blue
2         Blue
3         Blue
4         Blue
         ...  
16371    Green
16372     Blue
16373    Black
16374      NaN
16375      NaN
Name: EYE, Length: 16376, dtype: object

In [2969]:
df['HAIR'] = df['HAIR'].str.split(' ').str[0]
df['HAIR']

0        Brown
1        White
2        Black
3        Black
4        Blond
         ...  
16371       No
16372     Bald
16373     Bald
16374      NaN
16375      NaN
Name: HAIR, Length: 16376, dtype: object

In [2970]:
##SEX
df["SEX"] = df['SEX'].str.split(' ').str[0]
df["SEX"] 

0        Male
1        Male
2        Male
3        Male
4        Male
         ... 
16371    Male
16372    Male
16373    Male
16374    Male
16375     NaN
Name: SEX, Length: 16376, dtype: object

In [2971]:
#ID
df["ID"] = df['ID'].str.split(' ').str[0]
df["ID"]

0        Secret
1        Public
2        Public
3        Public
4            No
          ...  
16371        No
16372        No
16373    Secret
16374    Secret
16375       NaN
Name: ID, Length: 16376, dtype: object

In [2972]:
df["ID"] = df["ID"].replace(["No"], "None")
df["ID"]

0        Secret
1        Public
2        Public
3        Public
4          None
          ...  
16371      None
16372      None
16373    Secret
16374    Secret
16375       NaN
Name: ID, Length: 16376, dtype: object

* Some data is in the wrong format. For example, Year and APPEARANCES are floats but they really should be ints (as you cannot have a non-int number of appearances).

In [2973]:
#To convert to int
df['Year'] = df['Year'].astype('Int64')
df['Year']

0        1962
1        1941
2        1974
3        1963
4        1950
         ... 
16371    <NA>
16372    <NA>
16373    <NA>
16374    <NA>
16375    <NA>
Name: Year, Length: 16376, dtype: Int64

In [2974]:
df["APPEARANCES"]= df["APPEARANCES"].astype('Int64')  

In [2975]:
#FIRST APPEARANCE -- All record are matching with Year
all(
df[(df['FIRST APPEARANCE'].str.split('-',1).str[1] == df["Year"].astype(str).str[2:]) 
& (df["FIRST APPEARANCE"].notnull()) & (df["FIRST APPEARANCE"]!=u'')
& (df["Year"].notnull()) & (df["Year"]!=u'')].count()
    )

True

In [2976]:
#Same number on both sides not null and equal. See above.
df[(df["FIRST APPEARANCE"].notnull()) & (df["FIRST APPEARANCE"]!=u'')
& (df["Year"].notnull()) & (df["Year"]!=u'')][["FIRST APPEARANCE", "Year"]].count()

FIRST APPEARANCE    15561
Year                15561
dtype: int64

In [2977]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16376 entries, 0 to 16375
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   page_id           16376 non-null  int64 
 1   name              16376 non-null  object
 2   SecretIdentity    16376 non-null  object
 3   origin            2362 non-null   object
 4   urlslug           16376 non-null  object
 5   ID                12606 non-null  object
 6   ALIGN             13564 non-null  object
 7   EYE               6609 non-null   object
 8   HAIR              12112 non-null  object
 9   SEX               15522 non-null  object
 10  ALIVE             16373 non-null  object
 11  APPEARANCES       15280 non-null  Int64 
 12  FIRST APPEARANCE  15561 non-null  object
 13  Year              15561 non-null  Int64 
dtypes: Int64(2), int64(1), object(11)
memory usage: 1.8+ MB


In [2978]:
df.duplicated()#There is not duplicates according to this function

0        False
1        False
2        False
3        False
4        False
         ...  
16371    False
16372    False
16373    False
16374    False
16375    False
Length: 16376, dtype: bool

* Some column names are written as uppercase, some are lowercase and some are mixed case. Normalize them so all the column names are in the same format.


In [2979]:
df.fillna(0)

,page_id,name,SecretIdentity,origin,urlslug,ID,ALIGN,EYE,HAIR,SEX,ALIVE,APPEARANCES,FIRST APPEARANCE,Year
0,1678,Spider-Man,Spider-Man,0,http://marvel.fandom.com/wiki/Spider-Man_(Peter_Parker),Secret,Good,Hazel,Brown,Male,Living Characters,4043,Aug-62,1962
1,7139,Captain America,Captain America,0,http://marvel.fandom.com/wiki/Captain_America_(Steven_Rogers),Public,Good,Blue,White,Male,Living Characters,3360,Mar-41,1941
2,64786,Wolverine,Wolverine,0,http://marvel.fandom.com/wiki/Wolverine_(James_%22Logan%22_Howlett),Public,Neutral,Blue,Black,Male,Living Characters,3061,Oct-74,1974
3,1868,Iron Man,Iron Man,0,http://marvel.fandom.com/wiki/Iron_Man_(Anthony_%22Tony%22_Stark),Public,Good,Blue,Black,Male,Living Characters,2961,Mar-63,1963
4,2460,Thor,Thor,0,http://marvel.fandom.com/wiki/Thor_(Thor_Odinson),None,Good,Blue,Blond,Male,Living Characters,2258,Nov-50,1950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16371,657508,Ru'ach,Ru'ach,0,http://marvel.fandom.com/wiki/Ru%27ach_(Earth-616),None,Bad,Green,No,Male,Living Characters,0,0,0
16372,665474,Thane,Thane,Earth-616,http://marvel.fandom.com/wiki/Thane_(Thanos%27_son)_(Earth-616),None,Good,Blue,Bald,Male,Living Characters,0,0,0
16373,695217,Tinkerer,Tinkerer,Earth-616,http://marvel.fandom.com/wiki/Tinkerer_(Skrull)_(Earth-616),Secret,Bad,Black,Bald,Male,Living Characters,0,0,0
16374,708811,TK421,TK421,Earth-616,http://marvel.fandom.com/wiki/TK421_(Spiderling)_(Earth-616),Secret,Neutral,0,0,Male,Living Characters,0,0,0


In [2980]:
df.columns = map(str.lower, df.columns)

In [2981]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16376 entries, 0 to 16375
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   page_id           16376 non-null  int64 
 1   name              16376 non-null  object
 2   secretidentity    16376 non-null  object
 3   origin            2362 non-null   object
 4   urlslug           16376 non-null  object
 5   id                12606 non-null  object
 6   align             13564 non-null  object
 7   eye               6609 non-null   object
 8   hair              12112 non-null  object
 9   sex               15522 non-null  object
 10  alive             16373 non-null  object
 11  appearances       15280 non-null  Int64 
 12  first appearance  15561 non-null  object
 13  year              15561 non-null  Int64 
dtypes: Int64(2), int64(1), object(11)
memory usage: 1.8+ MB


In [2982]:
df[df["origin"].isna()]
#df["FIRST APPEARANCE"].notnull()) & (df["FIRST APPEARANCE"]!=u'')

,page_id,name,secretidentity,origin,urlslug,id,align,eye,hair,sex,alive,appearances,first appearance,year
0,1678,Spider-Man,Spider-Man,NaN,http://marvel.fandom.com/wiki/Spider-Man_(Peter_Parker),Secret,Good,Hazel,Brown,Male,Living Characters,4043,Aug-62,1962
1,7139,Captain America,Captain America,NaN,http://marvel.fandom.com/wiki/Captain_America_(Steven_Rogers),Public,Good,Blue,White,Male,Living Characters,3360,Mar-41,1941
2,64786,Wolverine,Wolverine,NaN,http://marvel.fandom.com/wiki/Wolverine_(James_%22Logan%22_Howlett),Public,Neutral,Blue,Black,Male,Living Characters,3061,Oct-74,1974
3,1868,Iron Man,Iron Man,NaN,http://marvel.fandom.com/wiki/Iron_Man_(Anthony_%22Tony%22_Stark),Public,Good,Blue,Black,Male,Living Characters,2961,Mar-63,1963
4,2460,Thor,Thor,NaN,http://marvel.fandom.com/wiki/Thor_(Thor_Odinson),None,Good,Blue,Blond,Male,Living Characters,2258,Nov-50,1950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16365,684262,K'thol,K'thol,NaN,http://marvel.fandom.com/wiki/K%27thol_(Earth-616),NaN,Good,NaN,NaN,Male,Deceased Characters,<NA>,NaN,<NA>
16369,660799,Melanie Kapoor,Melanie Kapoor,NaN,http://marvel.fandom.com/wiki/Melanie_Kapoor_(Earth-616),Public,Good,Blue,Black,Female,Living Characters,<NA>,NaN,<NA>
16370,674414,Phoenix's Shadow,Phoenix's Shadow,NaN,http://marvel.fandom.com/wiki/Phoenix%27s_Shadow_(Earth-616),NaN,Neutral,NaN,NaN,NaN,Living Characters,<NA>,NaN,<NA>
16371,657508,Ru'ach,Ru'ach,NaN,http://marvel.fandom.com/wiki/Ru%27ach_(Earth-616),None,Bad,Green,No,Male,Living Characters,<NA>,NaN,<NA>
